That last notebook got messy so I'm making a new cleaner one that will enable me to sample over both emulators simultaneously. 

In [201]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [202]:
import numpy as np
import h5py
from chainconsumer import ChainConsumer
#from corner import corner
from ast import literal_eval
from pearce.emulator import LemonPepperWet
from os import path
from scipy.linalg import inv

In [203]:
fname = '/u/ki/swmclau2/des/PearceMCMC/HOD_wp_ds_rmin_0.5_HOD.hdf5'

In [204]:
f = h5py.File(fname, 'r')

In [205]:
tf = f.attrs['training_file']
fixed_params = literal_eval(f.attrs['fixed_params'])
if 'rmin' in fixed_params:
    del fixed_params['rmin']
emu_hps = literal_eval(f.attrs['emu_hps'])

In [206]:
tf

array(['/u/ki/swmclau2/des/Aemulators/wp_zheng07_h/PearceWpCosmo.hdf5',
       '/u/ki/swmclau2/des/Aemulators/ds_zheng07_h/PearceDsCosmo.hdf5'],
      dtype='|S61')

In [207]:
fixed_params = fixed_params#[0]

In [208]:
emus = []
for t in tf:
    print t
    emus.append(LemonPepperWet(t, fixed_params = fixed_params, hyperparams = emu_hps) )

/u/ki/swmclau2/des/Aemulators/wp_zheng07_h/PearceWpCosmo.hdf5
3
/u/ki/swmclau2/des/Aemulators/ds_zheng07_h/PearceDsCosmo.hdf5
3


In [209]:
chain_pnames = f.attrs['param_names']

In [210]:
n_walkers = f.attrs['nwalkers']

In [211]:
n_burn = 0
chain = f['chain'][n_burn*n_walkers:, :]

In [212]:
rmin = 0#eval(f.attrs['fixed_params'])['rmin']

In [213]:
n_params = chain.shape[1] if len(chain.shape) > 1 else 1

In [214]:
print chain.shape, chain.shape[0]/n_walkers

(2989500, 12) 11958


In [215]:
c = ChainConsumer()
chain = chain.reshape((-1, n_walkers, chain.shape[1]))
chain = chain.reshape((-1, chain.shape[2]), order = 'F')
c.add_chain(chain, parameters=list(chain_pnames), walkers = n_walkers)


In [216]:
#MAP = chain.mean(axis = 0)
summary = c.analysis.get_summary(chains=0)
MAP = np.array([summary[key][1] for key in chain_pnames])
print MAP

[ 2.36316095e-02  1.12705331e-01 -5.68054402e-01  9.38044641e-01
  6.68610491e+01  2.65826548e+00  7.84735325e-01  1.42203106e+01
  1.32339055e+01  1.01569912e-01  1.11483999e+00  5.04712400e-01]


In [217]:
chain_pnames

array(['ombh2', 'omch2', 'w0', 'ns', 'H0', 'Neff', 'sigma8', 'logM1',
       'logM0', 'sigma_logM', 'alpha', 'conc_gal_bias'], dtype='|S13')

In [218]:
MAP_dict = dict(zip(chain_pnames, MAP))

In [219]:
#fixed_params = eval(f.attrs['chain_fixed_params'])
#fixed_params.update(eval(f.attrs['sim'])['cosmo_params'])

In [220]:
MAP_dict.update(fixed_params)

In [221]:
MAP_dict

{'H0': 66.86104912429899,
 'Neff': 2.6582654777740222,
 'alpha': 1.114839993771857,
 'conc_gal_bias': 0.50471240025388,
 'logM0': 13.233905532044076,
 'logM1': 14.220310568945216,
 'ns': 0.9380446407872876,
 'ombh2': 0.02363160949992332,
 'omch2': 0.11270533068923533,
 'sigma8': 0.7847353249870556,
 'sigma_logM': 0.10156991211563776,
 'w0': -0.5680544017034284,
 'z': 0.0}

In [222]:
MAP_pred = np.hstack([emu.emulate_wrt_r(MAP_dict, r_bin_centers=emu.scale_bin_centers).squeeze() for emu in emus])
MAP_pred = MAP_pred.reshape((len(emus), -1))

In [223]:
hod_idxs = np.array(range(7, len(chain_pnames)))
cosmo_idxs = np.array(range(7))

In [224]:
if chain.shape[1] == 7:
    cosmo_chain = chain
else:
    hod_chain = chain[:,7:]#[:, hod_idxs]
    cosmo_chain = chain[:,:7]#[:, cosmo_idxs]

In [225]:
from pearce.mocks import cat_dict
cosmo_params = {}

In [226]:
cosmo_params['boxno'] = 1
cosmo_params['realization'] = 1

In [227]:
cat = cat_dict['testbox'](**cosmo_params)#construct the specified catalog!

In [228]:
h = 0.6777
cosmo_pnames = ['ombh2', 'omch2', 'w0', 'ns', 'H0', 'Neff', 'sigma8']
cosmo_true_vals = np.array([0.048206*h**2, 0.307115*h**2-0.048206*h**2,\
                            -1, 0.9667, \
                                h*100, 3.046, 0.8228]) #mdpl2

cat_val_dict = dict(zip(cosmo_pnames, cosmo_true_vals))

In [ ]:
cat_val_dict

{'H0': 67.77,
 'Neff': 3.046,
 'ns': 0.9667,
 'ombh2': 0.022139921041739997,
 'omch2': 0.11891102387661,
 'sigma8': 0.8228,
 'w0': -1.0}

In [ ]:
true_param_dict = cat_val_dict.copy()
#hod_params = {'alpha': 1.083, 'conc_gal_bias': 1.0, 'logM0': 13.2,'logM1': 14.2, 'sigma_logM': 0.2}#sim_info['hod_params']
hod_params = {'alpha': 0.95, 'conc_gal_bias': 1.0, 'logM0': 13.2,'logM1': 14.5, 'sigma_logM': 0.2}#sim_info['hod_params']


hod_params['mean_occupation_centrals_assembias_param1'] = 0.0
hod_params['mean_occupation_satellites_assembias_param1'] =0.0
hod_params['mean_occupation_centrals_assembias_slope1'] = 0.0
hod_params['mean_occupation_satellites_assembias_slope1'] = 0.0

#for pname in emu.get_param_names():
#    if 'mean_occupation' in pname:
#        hod_params[pname] = 0.0

for hp, hv in hod_params.iteritems():
    if hp == 'logMmin':
        continue
    true_param_dict[hp] = hv

true_param_dict['conc_gal_bias'] = 1.0
true_pred = emu.emulate_wrt_r(true_param_dict)[0]

AssertionError: The input_params passed into get_data did not match those the Emu knows about.                                               It's possible fixed_params is missing a parameter, or you defined an extra one.                                               Additionally, orded_params could be wrong too!
Param mean_occupation_satellites_assembias_param1 was in the input but not the training data.


In [ ]:
true_param_dict

In [ ]:
true_data = f['data'][()]#.flatten()

In [ ]:
true_data = true_data.reshape((2, -1))

In [ ]:
plt.plot(emu.scale_bin_centers, true_data[0])
plt.plot(emu.scale_bin_centers, true_data[1])

plt.loglog();

In [ ]:
rbc = emu.scale_bin_centers#[-len(emu.scale_bin_centers):]

In [ ]:
cov = f['cov'][()]
yerr = np.sqrt(np.diag(cov)).reshape((len(emus), -1))

In [ ]:
fig = plt.figure(figsize = (12,5))
for i in xrange(len(emus)):
    plt.subplot(1,2,i+1)
    true_pred = emus[i].emulate_wrt_r(true_param_dict).squeeze()
    plt.errorbar(rbc, true_data[i], yerr=yerr[i], label = 'Data')
    plt.plot(rbc, 10**MAP_pred[i], label = 'MAP')
    plt.plot(rbc, 10**true_pred, label = 'Emu at Truth')

    plt.loglog()
    plt.legend(loc='best')
plt.show();

In [ ]:
fig = plt.figure(figsize = (12,5))
for i in xrange(len(emus)):
    plt.subplot(1,2,i+1)
    true_pred = emus[i].emulate_wrt_r(true_param_dict).squeeze()
    plt.errorbar(rbc, np.ones_like(true_data[i]), yerr=yerr[i]/true_data[i], label = 'Data')
    plt.plot(rbc, (10**MAP_pred[i])/true_data[i], label = 'MAP')
    plt.plot(rbc, (10**true_pred)/true_data[i], label = 'Emu at Truth')
    plt.legend(loc='best')
    plt.xscale('log')
plt.show();

In [ ]:
emu.obs

In [ ]:
rbc

In [ ]:
npart_aemulus = 1400**3
npart_mdpl2 = 3840**3

downsample_factor = 1e-2
npart_aemulus_ds = npart_aemulus*downsample_factor
print npart_aemulus_ds, npart_aemulus_ds/npart_mdpl2

In [ ]:
chain_pnames

In [ ]:
true_pred = np.hstack([emu.emulate_wrt_r(true_param_dict, r_bin_centers=emu.scale_bin_centers).squeeze() for emu in emus])
true_pred = true_pred.reshape((len(emus), -1))

In [ ]:
rmin_idxs = rbc > rmin

In [ ]:
cov_rmin_idxs = np.r_[rmin_idxs, rmin_idxs]

In [ ]:
print 'True Red. Chi2'
R = (10**true_pred[:, rmin_idxs].flatten()-true_data[:, rmin_idxs].flatten())

chi2 =  R.T.dot(inv(cov[cov_rmin_idxs][:,cov_rmin_idxs])).dot(R)
dof = len(chain_pnames)

print chi2/dof

In [ ]:
print 'Map Red. Chi2'
R = (10**MAP_pred[:, rmin_idxs].flatten()-true_data[:, rmin_idxs].flatten())

chi2 =  R.T.dot(inv(cov[cov_rmin_idxs][:,cov_rmin_idxs])).dot(R)
dof = len(chain_pnames)

print chi2/dof

In [ ]:
N = 10
cmap = sns.color_palette("BrBG_d", N)

In [ ]:
emu.get_param_names()

In [ ]:
fig = plt.figure(figsize=(15,6))
varied_pname = 'H0'
lower, upper = emu.get_param_bounds(varied_pname)

for i in xrange(len(emus)):
    plt.subplot(1,2,i+1)
    plt.errorbar(rbc, np.ones_like(true_data[i]), yerr=yerr[i]/true_data[i], label = 'Data')
    pred = emus[i].emulate_wrt_r(MAP_dict.copy()).squeeze()
    plt.plot(rbc, (10**pred)/true_data[i], label = 'Truth', color = 'k')    

    plt.xscale('log')
    
for c, val in zip(cmap, np.linspace(lower, upper, N) ):
    param_dict = MAP_dict.copy() #true_param_dict.copy()
    param_dict[varied_pname] = val

    for i in xrange(len(emus)):
        plt.subplot(1,2,i+1)
        pred = emus[i].emulate_wrt_r(param_dict).squeeze()
        plt.plot(rbc, (10**pred)/true_data[i], label = '%.3f'%val, color = c)
    
plt.legend(loc='best')
plt.show();

In [ ]:
emu_avg = 10**(emus[1].y.mean(axis=(0,1))+emu._y_mean)

In [ ]:
plt.plot(rbc, emu_avg/true_data[1])
plt.errorbar(rbc, np.ones_like(true_data[i]), yerr=yerr[i]/true_data[i])
plt.xscale('log')

In [ ]:
cat.load(1.0, HOD='zheng07', particles=True)

In [ ]:
from scipy.optimize import minimize_scalar
def add_logMmin(hod_params, cat, nd = 5e-4):

    hod_params['logMmin'] = 13.0 #initial guess
    #cat.populate(hod_params) #may be overkill, but will ensure params are written everywhere
    def func(logMmin, hod_params):
        hod_params.update({'logMmin':logMmin})
        return (cat.calc_analytic_nd(hod_params, min_ptcl=20) - nd)**2

    res = minimize_scalar(func, bounds = (12,16), args = (hod_params,),\
                          options = {'maxiter':100}, method = 'Bounded')

    # assuming this doens't fail
    print 'logMmin', res.x
    hod_params['logMmin'] = res.x

In [ ]:
add_logMmin(hod_params, cat)

In [ ]:
cat.populate(hod_params, min_ptcl = 20)

In [ ]:
r_bins = np.logspace(-1, 1.6, 19)

In [ ]:
print (r_bins[1:]+r_bins[:-1])/2.0

In [ ]:
emu.scale_bin_centers

In [ ]:
r_bins

In [ ]:
[ 0.1,  0.13945832,  0.19448624,  0.27122726,  0.37824899,
                0.52749971,  0.73564225,  1.02591437,  1.43072299,  1.99526231,
                2.7825594 ,  3.88051073,  5.41169527,  7.54705957, 10.52500285,
                14.67799268, 20.46968272, 28.54667663, 39.81071706]